# worksheet 12: Plotting with pandas, matplotlib and seaborn
- Analyze brain MRI images (DICOM) analysis using pydicom (https://pydicom.github.io/pydicom/dev/index.html)
- seaborn data visualization library (https://seaborn.pydata.org/)

In [ ]:
import pandas as pd
import numpy as np
import pydicom
from matplotlib import pyplot as plt
import seaborn as sns

# Analyzing brain MRI DICOM images
- Transcriptional Architecture of the Primate Neocortex” study (https://pubmed.ncbi.nlm.nih.gov/22445337/)
- download supplemental data from here for rhesus macaque https://www.blueprintnhpatlas.org/static/download
- lets analyze a DICOM file for donor 14349, available from the `MRI.zip`

In [ ]:
test_dicom = pydicom.dcmread('0014.dcm')

In [ ]:
# display the contents of the entire dataset
# test_dicom

In [ ]:
# access specific metadata elements using keywords
test_dicom.PatientName

In [ ]:
# can also set this to a value you want
test_dicom.PatientName = 'foo'

In [ ]:
test_dicom.get('PatientName', 'Not available')

In [ ]:
test_dicom.get('BodyPartExamined', 'Not available')

In [ ]:
test_dicom.get('AnatomicRegionSequence', 'Not available')

In [ ]:
image_data = test_dicom.pixel_array

In [ ]:
type(image_data)

In [ ]:
image_data

In [ ]:
image_data.shape

In [ ]:
image_data.ndim

In [ ]:
image_data[:3, 5:10]

In [ ]:
plt.imshow(image_data)
plt.show()

## pydicom built in datasets
-https://pydicom.github.io/pydicom/dev/reference/generated/pydicom.data.get_testdata_file.html#pydicom.data.get_testdata_file
-https://pydicom.github.io/pydicom/dev/auto_examples/input_output/plot_read_dicom.html#sphx-glr-auto-examples-input-output-plot-read-dicom-py

In [ ]:
from pydicom.data import get_testdata_files
filename = get_testdata_files("CT_small.dcm")[0]
ds = pydicom.dcmread(filename)

In [ ]:
metadata_elems = [elem for elem in ds]

In [ ]:
metadata_elems[:5]

In [ ]:
'BodyPartExamined' in metadata_elems

In [ ]:
print(f"SOP Class........: {ds.SOPClassUID} ({ds.SOPClassUID.name})")
print()

pat_name = ds.PatientName
print(f"Patient's Name...: {pat_name.family_comma_given()}")
print(f"Patient ID.......: {ds.PatientID}")
print(f"Modality.........: {ds.Modality}")
print(f"Study Date.......: {ds.StudyDate}")
print(f"Image size.......: {ds.Rows} x {ds.Columns}")
print(f"Pixel Spacing....: {ds.PixelSpacing}")

# use .get() if not sure the item exists, and want a default value if missing
print(f"Slice location...: {ds.get('SliceLocation', '(missing)')}")

In [ ]:
plt.imshow(ds.pixel_array, cmap=plt.cm.bone) 
plt.show()

### Extract DICOM metadata and save in a pandas df


In [ ]:
import os

In [ ]:
dicom_metadata = {
    'patient_id': [ds.get('PatientID', np.nan)],
    'filename': [os.path.basename(filename)],
    'modality': [ds.get('Modality', np.nan)],
    'mean_pixel_array': [ds.pixel_array.mean()] 
}

In [ ]:
dicom_metadata

In [ ]:
dicom_metadata_df = pd.DataFrame(dicom_metadata, index=['1CT1'])

In [ ]:
dicom_metadata_df

### simulate clinical metadata df

In [ ]:
clinical_metadata = {
    'patient_id': [ds.get('PatientID', np.nan)],
    'diagnosis': ['cancer'],
    'age': [56]
}

In [ ]:
clinical_metadata_df = pd.DataFrame(clinical_metadata, index=['1CT1'])

In [ ]:
clinical_metadata_df

## Joins
- simple join on 'patient_id'
- inner join

In [ ]:
pd.merge(
    left=dicom_metadata_df,
    right=clinical_metadata_df,
    on='patient_id',
    how='inner'
)

In [ ]:
pd.merge(
    left=dicom_metadata_df,
    right=clinical_metadata_df,
    on='patient_id',
    indicator=True
)

### left join
- use keys from left frame only

In [ ]:
dicom_metadata['patient_id'].append('patient2')
dicom_metadata['filename'].append('patient2_file.dcm')
dicom_metadata['modality'].append('CT')
dicom_metadata['mean_pixel_array'].append(np.float64(100.2))

In [ ]:
dicom_metadata

In [ ]:
dicom_metadata_df = pd.DataFrame(
    dicom_metadata, index=['1CT1', 'patient2']
)

In [ ]:
pd.merge(
    left=dicom_metadata_df,
    right=clinical_metadata_df,
    on='patient_id',
    how='left',
    indicator=True
)

### right join
- use keys from right frame only

In [ ]:
pd.merge(
    left=dicom_metadata_df,
    right=clinical_metadata_df,
    on='patient_id',
    how='right',
    indicator=True
)

In [ ]:
clinical_metadata['patient_id'].append('patient3')
clinical_metadata['diagnosis'].append('normal')
clinical_metadata['age'].append('75')

In [ ]:
clinical_metadata

In [ ]:
clinical_metadata_df = pd.DataFrame(clinical_metadata, 
                                    index=['1CT1', 'patient3']
                                   )

In [ ]:
pd.merge(
    left=dicom_metadata_df,
    right=clinical_metadata_df,
    on='patient_id',
    how='right',
    indicator=True
)

### Numpy array manipulations
- normalize pixel array values (e.g. min-max normalization) and plot
- mask and only plot intensities above a certain value
- crop a region of interest from the pixel array and plot
### Dataset manipulations
- create custom tags for study description and group by this clinical metadata
- plot images by metadata tag

## Plotting with seaborn
- https://seaborn.pydata.org/

In [ ]:
gene_tx_df = pd.read_csv('genes_transcripts.csv')

### scatterplot 
- `tx_length` for each `gene_name` colored by `biotype`

In [ ]:
sns.scatterplot(x='gene_name', y='tx_length', data=gene_tx_df, hue='biotype')

### integrate with matplotlib
- move legend

In [ ]:
fig, ax = plt.subplots()
ax = sns.scatterplot(x='gene_name', y='tx_length', data=gene_tx_df, hue='biotype')
ax.legend(bbox_to_anchor=(1,1))
plt.show()

### `sns.pairplot`
- quickly inspect data distributions by facets

In [ ]:
sns.pairplot(gene_tx_df, hue='gene_name')

### Facet grid
faceting histograms by subsets of data using displot
- visualize univariate or bivariate dist of data
- hist
- kde
- ecdf

In [ ]:
sns.displot(data=gene_tx_df, x='tx_length', 
            col='gene_name', height=4)

#### add a KDE curve
- kernel density estimate/PDF

In [ ]:
sns.displot(data=gene_tx_df, x='tx_length', 
            col='gene_name', height=4, kde=True)

### boxplots

In [ ]:
fig, ax = plt.subplots()
ax = sns.boxplot(x='gene_name', y='tx_length', hue='biotype', data=gene_tx_df)
ax.legend(bbox_to_anchor=(1,1))
plt.show()


### histplot

In [ ]:
fig, ax = plt.subplots()
ax = sns.histplot(data=gene_tx_df, x='tx_length', hue='gene_name')
plt.show()

### stripplots

In [ ]:
fig, ax = plt.subplots()
ax = sns.stripplot(x='gene_name', y='tx_length', hue='biotype', data=gene_tx_df)
ax.legend(bbox_to_anchor=(1, 1))
plt.show()

### variety of public datasets available in seaborn for exploration

In [ ]:
sns.get_dataset_names()

# Collaborative Exercise

## Exercise 1
fastMRI exploration

- Seaborn has provided a sample dataset of fMRI data (functional magnetic resource imaging data)
- Good exercise for exploring different types of plots and functionality in seaborn
- You will explore this dataset and try to plot the different features
- Load the data using `sns.load_dataset('fmri')`
- Can you describe this data?
- Can you figure out different ways to plot the features of this dataset using seaborn? For e.g. lineplots/relplots for timepoints versus signal that are colored by region and styled by event, boxplots and swarmplots of the same data
- Is there a linear or non-linear relationship between timepoint and signals for the different regions? What functionality can you use from seaborn to explore this?
